## Build a Dashboard Application with Plotly Dash

In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in real-time.

This dashboard application contains input components such as a dropdown list and a range slider to interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

TASK 1: Add a Launch Site Drop-down Input Component

TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown

TASK 3: Add a Range Slider to Select Payload

TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot

Your completed dashboard application should look like the following screenshot:



After visual analysis using the dashboard, you should be able to obtain some insights to answer the following five questions:

Which site has the largest successful launches?

Which site has the highest launch success rate?

Which payload range(s) has the highest launch success rate?

Which payload range(s) has the lowest launch success rate?

Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest\ launch success rate?

Estimated time needed: 90 minutes


##### Important Notice about this lab environment

In [1]:
# Import required libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

In [2]:
airline_data =  pd.read_csv('C:/Users/wli161606/OneDrive - Applied Materials/Desktop/Algorithm/Python/10. Data Science capstone/spacex_launch_dash.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})

In [3]:
min_payload=airline_data['Payload Mass (kg)'].min()
max_payload=airline_data['Payload Mass (kg)'].max()
print(min_payload,max_payload)

0.0 9600.0


In [ ]:
# Create a dash application
# app = JupyterDash(__name__)
# JupyterDash.infer_jupyter_proxy_config()

# Create a dash application
app = dash.Dash(__name__)

# Build dash app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard', 
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 30}),
                                dcc.Dropdown(id='site_dropdown',
                                             options=[
                                                 {'label': 'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                                 {'label': 'CCAFS SLC-40','value':'CCAFS SLC-40'},
                                                 {'label': 'CCAFSSLC-40','value':'CCAFSSLC-40'},
                                                 {'label': 'KSC LC-39A','value':'KSC LC-39A'},
                                                 {'label': 'VAFB SLC-4E','value':'VAFB SLC-4E'},
                                                 {'label': 'ALL','value':'ALL'}],
                                            value=['ALL'],
                                            placeholder='Select a Launch Site here', 
                                            searchable=True),      
                                html.Br(),
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                
                                html.Br(),
                                html.P("Payload range (Kg):"),
                                dcc.RangeSlider(id='payload-slider',min=0, max=10000,step=1000, value=[2000,8000]),
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                                ])

    
@app.callback(
    Output(component_id='success-pie-chart',component_property='figure'),
    [Input(component_id='site_dropdown', component_property='value')],
    prevent_initial_call=True)


def update_pie(selected_site):
    
    if selected_site in ['CCAFS LC-40','CCAFS SLC-40','CCAFSSLC-40','KSC LC-39A','VAFB SLC-4E']:
        df_pie = airline_data[airline_data['Launch Site']==selected_site]
        fig_pie=px.pie(df_pie,names='class', title='success-pie-chart')
        fig_pie.update_layout()
        return fig_pie
    
    elif selected_site in ['ALL']:
        fig_pie=px.pie(airline_data,values='class',names='Launch Site', title='success-pie-chart')
        fig_pie.update_layout()
        return fig_pie
#-----------------------------------------------------------------------------------------------------    
@app.callback(
    Output(component_id='success-payload-scatter-chart',component_property='figure'),
    [Input(component_id='payload-slider', component_property='value'),
    Input(component_id='site_dropdown', component_property='value')],
    prevent_initial_call=True)


def update_scatter(selected_payload,selected_site): 
    if selected_site in ['CCAFS LC-40','CCAFS SLC-40','CCAFSSLC-40','KSC LC-39A','VAFB SLC-4E']:
        df = airline_data[airline_data['Launch Site']==selected_site]
        df_slider=df[(df['Payload Mass (kg)']<=selected_payload[1]) &(df['Payload Mass (kg)']>=selected_payload[0])]
        fig_scatter=px.scatter(df_slider,x='Payload Mass (kg)',y='class', title='success-payload-scatter-chart',color='Booster Version Category')
        fig_scatter.update_layout()
        return fig_scatter
    
    elif selected_site in ['ALL']:
        df_slider=airline_data[(airline_data['Payload Mass (kg)']<=selected_payload[1]) &(airline_data['Payload Mass (kg)']>=selected_payload[0])]
        fig_scatter=px.scatter(df_slider,x='Payload Mass (kg)',y='class', title='success-payload-scatter-chart',color='Booster Version Category')
        fig_scatter.update_layout()
        return fig_scatter
    
# Run the app
if __name__ == '__main__':
    app.run_server()    

#------------------------------------------------------------------------------------------------------------------#
# def run_server(self,
#                port=8050,
#                debug=True,
#                threaded=True,
#                **flask_run_options):
#     self.server.run(port=port, debug=debug, **flask_run_options)

# if __name__ == '__main__':
#     app.run_server(mode="inline", host="localhost", debug=True, port=8050) # or whatever you choose